In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-2-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
############################################
################# Load data ################
############################################

# #
# def get_sessions_from_directory_listing(root_dir,
#                                        animal_id):
    
#     sessions = os.listdir(root_dir+animal_id)
#     print ("sessions: ", sessions)
    
#     return sessions
    
#

animal_ids = [
    'DON-011733',
    #'DON-002865',
    #'DON-003165'    
    #'DON-003343'
    #'DON-006084'
    #'DON-006085'
    #'DON-006087',
    #'DON-009192',
    #'DON-010473',
    #'DON-010477'
]
              
#
root_dir = '/media/cat/4TB/donato/'
root_dir = '/home/cat/data/bmi/'
animal_id = animal_ids[0]
dir_ = '/002P-F/tif/'

#session = 'DON-002865_202102116'
#session = 'DON-003343_20210220'
session = '20230309'
#session = '20210601'
#sessions = get_sessions_from_directory_listing(root_dir,
#                                              animal_id)

print ("PRocessing: ", animal_id)
#
#
print ("processing: ", session)
c = calcium.Calcium()
c.root_dir = root_dir
c.data_dir = os.path.join(root_dir, 
                          animal_id, 
                          session
                          #+dir_
                          , 
                          #'suite2p',
                          'plane0')
c.animal_id = animal_id
c.session = session
c.detrend_model_order = 1

#
c.load_suite2p()
c.save_python = True
c.save_matlab = False

# 
c.load_binarization()
binarization_method = 'upphase'
if binarization_method=='onphase':
    traces = c.F_onphase_bin
elif binarization_method=='upphase':
    traces = c.F_upphase_bin
else:
    print ("METHOD NOT FOUND")

# print ("binarized data: ", traces.shape)

###################################################################
########## cleanup cells + compute pairwise correlations ##########
###################################################################
c.subselect_moving_only = False
c.subselect_quiescent_only = False
c.load_footprints()
# c.corr_parallel_flag = True
# c.zscore = True                     # here we compute correlations based on N shuffles of the data and zscore
#                                     # otherwise we save the raw correlation also
# c.n_tests_zscore = 1000

# #

# # 
# c.n_cores = 32
# c.recompute_correlation = False
# c.binning_window = 30        # binning window in frames
# c.subsample = 1              # subsample traces by this factor
# c.scale_by_DFF = True        # scale traces by DFF
# c.compute_correlations()

# remove duplicate neurons requires correlation array to be computed first; it uses physical overalp but also correlation threshold

#
print ("... NOTE: dedupliation method should be generally used with correlation data ... ")
print ("     standalone version here is for testing purposes only ... ")
c.zscore = False
c.shuffle_data = False
c.deduplication_use_correlations = False
c.correlation_datatype = 'upphase'      # filtered vs. upphase

# 
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max overlap for overlap method
c.corr_threshold = 0.3                  # max correlation allowed for high overlap
c.zscore_threshold = 3.0                # zscore threshold for high overlap

#  
c.corr_delete_method = 'highest_connected_no_corr' #'highest_connected', lowest_snr', 'highest_connected_no_corr' <- this skips correlation check
c.recompute_deduplication = True
c.recompute_overlap = False    #this is not required to be redone, it's just once
c.make_correlation_dirs()
c.remove_duplicate_neurons()     # this removes duplicate neurons and saves non-duplicate version of correlation array

#
print ("DONE")

#


PRocessing:  DON-011733
processing:  20230309
... NOTE: dedupliation method should be generally used with correlation data ... 
     standalone version here is for testing purposes only ... 
Removed cells:  33
number of contours: 299
DONE


In [65]:
D = {
    'data_dir': 'test1',
}

# extract value for key "data_dir" from dictionary D
data_dir = D['data_dir']
print (data_dir)

    

test1


In [ ]:
#d = np.load('/media/cat/4TB/donato/DON-002865/DON-002865_20210210/002P-F/tif/suite2p/plane0/allcell_correlation_array_filtered.npy')
#d = np.load('/media/cat/4TB/donato/DON-002865/DON-002865_20210210/002P-F/tif/suite2p/plane0/allcell_correlation_array_upphase.npy')
d = np.load('/media/cat/4TB/donato/DON-002865/DON-002865_20210210/002P-F/tif/suite2p/plane0/goodcell_correlations_array_post_deduplication_filtered.npy')
print (d.shape)

plt.figure()
plt.imshow(d[:,:])
plt.colorbar()

plt.show()

(806, 806)


In [ ]:
def get_corr2(temp1, temp2, dynamic_corr_flag, n_tests=1000):
    # 
    # check if all values are the same
    if np.all(temp1==temp1[0]):
        corr = [np.nan,1]
        return corr, [np.nan], [np.nan]

    if np.all(temp2==temp2[0]):
        corr = [np.nan,1]
        return corr, [np.nan], [np.nan]

    # if using dynamic correlation we need to compute the correlation for 1000 shuffles
    corr_array = []
    corr = scipy.stats.pearsonr(temp1, temp2)
    corr_array.append(corr[0])
                                

    if dynamic_corr_flag:
        corr_s = []
        for k in range(n_tests):
            # choose a random index ranging from 0 to the length of the array minus 1
            idx = np.random.randint(-temp2.shape[0], temp2.shape[0],1)
            #idx = np.random.randint(temp2.shape[0])
            temp2_shuffled = np.roll(temp2, idx)
            corr_s = scipy.stats.pearsonr(temp1, temp2_shuffled)
            corr_array.append(corr_s[0])

        # compute z-score
        corr_z = stats.zscore(corr_array)

        #
        corr = [corr[0], corr_z[0]]
        
    return corr, corr_z, corr_array

####################################
data  = np.load('/media/cat/4TB/donato/DON-006084/20210523/002P-F/tif/suite2p/plane0/binarized_traces.npz',
                allow_pickle=True)
from scipy import stats
#
subsample = 5
dynamic_corr_flag = True
scale_by_DFF = True
rasters_DFF = data['F_upphase']
DFF = data['DFF']
binning_window = 30

#
k = 1
temp1 = rasters[k][::subsample]

# scale by rasters_DFF
if scale_by_DFF:
    temp1 = temp1*rasters_DFF[0][::subsample]

# bin data in chunks of size binning_window
if binning_window!=1:
    tt = []
    for q in range(0, temp1.shape[0], binning_window):
        temp = np.sum(temp1[q:q + binning_window])
        tt.append(temp)
    temp1 = np.array(tt)

#
cz = []
from tqdm import trange
for p in trange(500):
    temp2 = rasters[p][::subsample]

    # scale by rasters_DFF
    if scale_by_DFF:
        temp2 = temp2*rasters_DFF[p][::subsample]

    # 
    if binning_window!=1:
        tt = []
        for q in range(0, temp2.shape[0], binning_window):
            temp = np.sum(temp2[q:q + binning_window])
            tt.append(temp)
        temp2 = np.array(tt)

    #
    corr, corr_z, corr_array = get_corr2(temp1, temp2, dynamic_corr_flag)

    #print ("corr: ", corr)
    cz.append(corr_z[0])

# plot cz
plt.figure()
plt.plot(cz)
plt.show()




# ####################################
# plt.figure()

# # plot histogram of corr_z
# plt.subplot(121)
# plt.hist(corr_z, bins=50)

# # smae for corr_array
# plt.subplot(122)
# plt.hist(corr_array, bins=50)
# plt.plot([corr[0],corr[0]],[0,100],'r--',linewidth=3)

# #plt.semilogy()
# plt.show()

100%|██████████| 500/500 [00:14<00:00, 34.30it/s]


In [ ]:
# sort cz and plot
cz = np.array(cz)
idx = np.argsort(cz)[::-1]
print (idx.shape)

plt.figure()
plt.plot(cz[idx])
plt.show()


(500,)


In [94]:
def load_ttl_and_video_times_absolute(fname_results,
                                      fname_video):

    d = np.load(fname_results, allow_pickle=True)

    ttl_times = d['ttl_times']

    vid = np.load(fname_video, allow_pickle=True)

    vid_times = vid['vide_frame_times']

    #print (vid_times)

    vid_times = vid_times[:,1]
    vid_times = vid_times - vid_times[0]+ttl_times[0]

    print ("#ttl times: ", ttl_times.shape)
    print ("#Video times: ", times.shape)

    return ttl_times, vid_times


#
fname_results = '/home/cat/data/bmi/DON-011733/20230309/results.npz'
fname_video = '/home/cat/data/bmi/DON-011733/20230309/video_data.npz'

ttl_times, vid_times = load_ttl_and_video_times_absolute(fname_results,
                                                            fname_video)




#ttl times:  (90000,)
#Video times:  (89155,)


[  32.7636478    32.7895813    32.82049561 ... 3033.96480322 3034.00370026
 3034.03162456]
(89155,)
